In [26]:
include("../src/Julia.jl")

pcgWorker (generic function with 1 method)

In [63]:
gname = "../../graphs/rand_iu_1000_3/"
wmax = 1000
iu = true
n = 1000;
p = 0.02;

In [64]:
function wgen()
    if iu
        return 1 / rand(1:wmax)
    else
        return rand(1:wmax)
    end
end

wgen (generic function with 1 method)

In [65]:
a = randGraph(n, p=p, weightGen = wgen);
ceil(Int64,nnz(a)/2)

9913

In [66]:
tree1 = akpw(a);
str1 = compStretches(tree1,a);
data1 = sum(str1.nzval)/2,mean(str1.nzval)

(18944.06281332134,1.9110322620116351)

In [67]:
tree2 = randishPrim(a);
str2 = compStretches(tree2,a);
data2 = sum(str2.nzval)/2,mean(str2.nzval)

(98158.44351338221,9.901991678945043)

In [68]:
tree3 = randishKruskal(a);
str3 = compStretches(tree3,a);
data3 = sum(str3.nzval)/2,mean(str3.nzval)

(239712.62476036008,24.18164276811864)

In [69]:
truex = randn(n); truex = truex - mean(truex);
b = lap(a) * truex; b = b - mean(b);

In [70]:
writeToFile(gname * "graph.mtx", a);

In [71]:
writeToFile(gname * "tree1.mtx", tree1);
writeToFile(gname * "tree2.mtx", tree2);
writeToFile(gname * "tree3.mtx", tree3);

In [72]:
writeToFile(gname * "x.vec", truex);
writeToFile(gname * "b.vec", b);

In [73]:
function strDist{Tv,Ti}(tr::SparseMatrixCSC{Tv,Ti}, a::SparseMatrixCSC{Tv,Ti})
    
    res = Array{ASCIIString,1}(0)
    
    str = compStretches(tr, a-tr).nzval;
    
    vmax = maximum(str)
    inter = (0,1)
    while inter[2] < vmax
        cnt = 0
        for i in 1:length(str)
            if inter[1] <= str[i] && str[i] < inter[2]
                cnt = cnt + 1
            end
        end
        # don't count edges twice
        cnt = ceil(Int64,cnt / 2)
        
        push!(res, "($(inter[1]),$(inter[2])) - $(cnt)")
        
        inter = (inter[2], inter[2] * 2)
    end
    
    push!(res, "")
    
    return res;
    
end

strDist (generic function with 1 method)

In [74]:
output = []
push!(output, "A random graph with $(n) vertices and $(ceil(Int64,nnz(a)/2)) edges.")
push!(output, "")
push!(output, "The graph is generated in the following way:")
push!(output, "    - for each vertex, select n / $(1/p) random neighbors")
push!(output, "    - get rid of duplicate edges")
if iu
    push!(output, "    - weights are reals sampled uniformly from 1/[1..$(wmax)]")
else
    push!(output, "    - weights are integers sampled uniformly from [1..$(wmax)]")
end
push!(output, "")
push!(output, "There are three trees in the folder.")
push!(output, "")
push!(output, "Tree1 - total stretch = $(data1[1]), ave stretch = $(data1[2])")
append!(output, strDist(tree1,a))
push!(output, "Tree2 - total stretch = $(data2[1]), ave stretch = $(data2[2])")
append!(output, strDist(tree2,a))
push!(output, "Tree3 - total stretch = $(data3[1]), ave stretch = $(data3[2])")
append!(output, strDist(tree3,a))

desc = open(gname * "description.txt", "w")
for i in 1:length(output)
    println(desc, output[i])
end
close(desc)